## Monada Result
- ### Funkcyjna alternatywa dla wyjątków
- ### Służy do wyrażania i traktowania niepowodzenia w aplikacji w postaci zwykłej wartości
- ### W przeciwieństwie do wyjątków jest jawne, zmuszając tym samym programistę do obsługi porażki
- ### W niektórych językach istnieje również pod nazwą Try, z wariantami Success i Failure

### Abstrakcyjny typ Result ma dwa warianty:
- ### Ok - reprezentujący wartość pochodzącą z operacji która się powiodła
- ### Error - reprezentujący błąd powstały w trakcie wykonywania operacji - odpowednik wyjątku

## Podstawowy przykład

### Bez użycia Result

In [ ]:
def divide(x: float, y: float) -> float:
    return x / y

value_pairs = [(3, 2), (1, 0), (3.1, 0.0), (0, 3)]

divided_results = [divide(x, y) for x, y in value_pairs]
divided_results

### Używając Result

In [ ]:
import result
from result import Result, Error, Ok

def divide(x: float, y: float) -> Result[float, str]:
    if y == 0:
        return Error("Divisor cannot be 0")
    return Ok(x / y)

value_pairs = [(3, 2), (1, 0), (3.1, 0.0), (0, 3)]

divided_results = [divide(x, y) for x, y in value_pairs]
divided_results

## Agregacja

In [ ]:
from typing import List
import result
from result import Result, Error, Ok

def divide(x: float, y: float) -> Result[float, List[str]]:
    if y == 0:
        return Error(["Divisor cannot be 0"])
    return Ok(x / y)

# value_pairs = [(3, 2), (1, 0), (3.1, 0), (0, 3)]
value_pairs = [(3, 2), (1, 1), (3.1, 0.1), (0, 3)]

divided_results = [divide(x, y) for x, y in value_pairs]
print(divided_results)

agg = result.aggregate(divided_results)
agg

## Złożony przykład

In [ ]:
divide_with_context = lambda x, y: divide(x, y).map_error(lambda e: [f"DZIELENIE {x}/{y}: {e[0]}"])

@result.from_generator
def divide_and_report(x1, y1, x2, y2):
    divided_1 = yield divide_with_context(x1, y1)
    divided_2 = yield divide_with_context(x2, y2)
    return f"PODSUMOWANIE: {divided_1}, {divided_2}"

divide_and_report(3, 1, 2, 1)

In [ ]:
divide_with_context = lambda x, y: divide(x, y).map_error(lambda e: [f"DZIELENIE {x}/{y}: {e[0]}"])

@result.from_generator
def divide_and_report_aggregate(x1, y1, x2, y2):
    divided_1, divided_2 = yield result.aggregate([
        divide_with_context(x1, y1),
        divide_with_context(x2, y2)
    ])
    return f"PODSUMOWANIE: {divided_1}, {divided_2}"

divide_and_report_aggregate(3, 1, 2, 1)

## Rozpakowywanie wartości z Result


### [Bezpieczne] result_var.match - wymaga obsługi obu przypadków


In [ ]:
# value = Ok(42)
value = Error("tragedia")

unpacked_value = value.match(
    lambda ok_val: f"Wartość jest ok: {ok_val}",
    lambda error_val: f"Wartość jest błędna: {error_val}",
)
unpacked_value

### [Bezpieczne] map/flat_map/yield - kontynuacja działania wewnątrz monady


In [ ]:
value = Ok(42)
# value = Error("tragedia")

print(value.map_ok(lambda x: x + 58))

def continuation_fn(x):
    if x != 42:
        return Ok(x + 58)
    else:
        return Error("x nie może być równe 42")

print(value.flat_map(continuation_fn))

### [Niebezpieczne] result_var.try_get() - rozpakowanie "na siłę", rzuca wyjątek gdy result_var jest typu Error

In [ ]:
# value = Ok(42)
value = Error("tragedia")

value.try_get()